# 数据集：人民日报1998年中文标注语料库
# 目的：分词

In [2]:
import numpy as np
from tqdm import tqdm

In [11]:
def trainParameter(fileName):
    """
    根据文本训练HMM模型参数PI、A、B
    Q: 状态集合
    V: 观测集合
    A: 状态转移概率矩阵
    B: 观测概率矩阵
    O: 观测序列
    PI: 初始概率分布
    """
    
    # 定义一个查询字典，用于映射四种标记在数组中的对应的位置
    # B：词语的开头 M：一个词语的中间词 E：一个词语的结果 S：非词语，单个词
    # 如：我 非常地 爱 中国
    #       S  BME    S  BE
    
    statuDict = {'B': 0, 'M': 1, 'E': 2, 'S': 3}
    
    # 每个字有四种状态
    # 初始化PI数组->对应四种状态
    PI = np.zeros(4)
    
    # 初始化状态转移矩阵
    A = np.zeros((4, 4))
    # 初始化观测概率
    # 中文分词，ord(汉字)->对应汉字编码，用65536空间保证所有汉字能存储到
    B = np.zeros((4, 65536))
    
    # 读训练文本
    fr = open(fileName, encoding = 'utf-8')
    
    
    # 在文件中，每一行为一个样本
    for line in tqdm(fr.readlines()):
        currentLine = line.strip().split()
        # 词的状态标记
        wordLabel = []
        
        # 对每一个词遍历
        for i in range(len(currentLine)):
            if len(currentLine[i]) == 1:
                label = 'S'
            else:
                label = 'B' + 'M' * (len(currentLine[i]) - 2) + 'E'
                
            if i == 0:
                PI[statuDict[label[0]]] += 1
            
            # 对于该单词，统计与状态链B
            for j in range(len(label)):
                B[statuDict[label[j]]][ord(currentLine[i][j])] += 1
                
            wordLabel.extend(label)
            
        for i in range(1, len(wordLabel)):
            A[statuDict[wordLabel[i - 1]]][statuDict[wordLabel[i]]] += 1
            
    # 对PI求和
    sum_ = np.sum(PI)
    for i in range(len(PI)):
        if PI[i] == 0:
            PI[i] = -3.14e+100
        else:
            PI[i] = np.log(PI[i] / sum_)
    
    # 对A求概率log
    for i in range(len(A)):
        sum_ = np.sum(A[i])
        for j in range(len(A[i])):
            if A[i][j] == 0:
                A[i][j] = -3.14e+100
            else:
                A[i][j] = np.log(A[i][j] / sum_)
    
    # 对B求概率log
    for i in range(len(B)):
        sum_ = np.sum(B[i])
        for j in range(len(B[i])):
            if B[i][j] == 0:
                B[i][j] = -3.14e+100
            else:
                B[i][j] = np.log(B[i][j] / sum_)
                
    return PI, A, B

def loadArticle(fileName):
    """
    加载文章
    """
    article = []
    fr = open(fileName, encoding='utf-8')
    for line in tqdm(fr.readlines()):
        line = line.strip()
        article.append(line)
    
    return article

def participle(article, PI, A, B):
    """
    分词
    维特比算法
    """
    
    # 初始化文章列表
    retAritcle = []
    
    for line in article:
        #
        delta = [[0 for i in range(4)] for i in range(len(line))]
        
        for i in range(4):
            delta[0][i] = PI[i] + B[i][ord(line[0])]
            
        psi = [[0 for i in range(4)] for i in range(len(line))]
        
        # 递推
        for t in range(1, len(line)):
            for i in range(4):
                tempDelta = [0] * 4
                for j in range(4):
                    tempDelta[j] = delta[t - 1][j] + A[j][i]
                    
                maxDelta = max(tempDelta)
                maxDeltaIndex = tempDelta.index(maxDelta)
                
                delta[t][i] = maxDelta + B[i][ord(line[t])]
                
                psi[t][i] = maxDeltaIndex
                
        
        # 状态链列表
        sequence = []
        # 第三步：终止
        i_opt = delta[len(line) - 1].index(max(delta[len(line) - 1]))
        
        sequence.append(i_opt)
        
        # 第四步：最优路径回溯
        for t in range(len(line) - 1, 0, -1):
            i_opt = psi[t][i_opt]
            sequence.append(i_opt)
        sequence.reverse()
        
        # 开始分词
        curLine = ''
        # 遍历该行每一个词
        for i in range(len(line)):
            curLine += line[i]
            if (sequence[i] == 3 or sequence[i] == 2) and i != (len(line) - 1):
                curLine += '|'
                
        retAritcle.append(curLine)
        
    return retAritcle


In [13]:
# train hmm parmeter
PI, A, B = trainParameter('./hmm_data/HMMTrainSet.txt')
# test
article = loadArticle('./hmm_data/testArtical.txt')

# 原文
print("====== 打印原文 ======")
for line in article:
    print(line)
    
# 分词
partiArticle = participle(article, PI, A, B)

# 分词后结果
print("\n====== 分词后 ======")
for line in partiArticle:
    print(line)


100%|██████████████████████████████████████████| 3/3 [00:00<00:00, 53544.31it/s]

====== 打印原文 ======
深圳有个打工者阅览室
去年１２月，我在广东深圳市出差，听说南山区工商分局为打工者建了个免费图书阅览室，这件新鲜事引起了我的兴趣。
１２月１８日下午，我来到了这个阅览室。阅览室位于桂庙，临南油大道，是一间轻体房，面积约有４０平方米，内部装修得整洁干净，四周的书架上摆满了书，并按政治、哲学、法律法规、文化教育、经济、科技、艺术、中国文学、外国文学等分类，屋中央有两排书架，上面也摆满了图书和杂志。一些打工青年或站或蹲，认真地阅读，不时有人到借阅台前办理借书或还书手续。南山区在深圳市西边，地处城乡结合部，外来打工者较多。去年２月，南山区工商分局局长王安全发现分局对面的公园里常有不少打工者业余时间闲逛，有时还滋扰生事。为了给这些打工者提供一个充实自己的场所，他提议由全分局工作人员捐款，兴建一个免费阅览室。领导带头，群众响应，大家捐款１．４万元，购买了近千册图书。３月６日，建在南头繁华的南新路和金鸡路交叉口的阅览室开放了。从此，这里每天都吸引了众多借书、看书的人们，其中不仅有打工者，还有机关干部、公司职员和个体户。到了夏天，由于阅览室所在地被工程征用，南山区工商分局便把阅览室迁到了桂庙。阅览室的管理人员是两名青年，男的叫张攀，女的叫赵阳。张攀自己就是湖北来的打工者，听说南山区工商分局办免费阅览室，便主动应聘来服务。阅览室每天从早９时开到晚１０时，夜里张攀就住在这里。他谈起阅览室里的图书，翻着一本本的借阅名册，如数家珍，对图书和工作的挚爱之情溢于言表。我在这里碰到南山区华英大厦一位叫聂煜的女青年，她说她也是个打工者，由于春节探家回来后就要去市内工作，很留恋这里的这个免费阅览室，想抓紧时间多看些书，她还把自己买的几本杂志捐给了阅览室。在阅览室的捐书登记簿上，记录着这样的数字：工商系统内部捐书３５５０册，社会各界捐书２５０册。我在阅览室读到了这样几封感谢信：深圳瑞兴光学厂的王志明写道：“我们这些年轻人远离了家乡，来到繁华紧张的都市打工，辛劳之余，能有机会看书读报，感到特别充实。”深圳文光灯泡厂的江虹说：“南山区工商分局的干部职工捐款、捐书，给我们打工者提供良好的学习环境，鼓励我们求知上进，真是办了一件大好事，他们是我们打工者的知音。”（本报记者罗华）

====== 分词后 ======
深圳|有个|打|工者|阅览室
去年|１２月|，|我|在|广